In [29]:
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

In [3]:
train_set = pd.read_csv("train.csv", delimiter=",", parse_dates=True)
predict_set = pd.read_csv("prediction.csv", delimiter=",", parse_dates=True)

In [4]:
feature_list = ["Keyword_ID","Ad_group_ID","Campaign_ID","Account_ID","Device_ID","Match_type_ID"]

In [5]:
train_set[feature_list].astype(str).describe()

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID
count,8285423,8285423,8285423,8285423,8285423,8285423
unique,487981,269480,2927,16,3,3
top,981509692733,428124648205,919168440848,604905316813,298643508640,95725474456
freq,5188,8111,325658,2300896,3980401,3754784


In [6]:
predict_set[feature_list].astype(str).describe()

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID
count,578012,578012,578012,578012,578012,578012
unique,195748,118978,2259,16,3,3
top,981509692733,501719917851,919168440848,604905316813,298643508640,95725474456
freq,359,519,23375,176184,283650,264997


In [7]:
train_set[feature_list].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8285423 entries, 0 to 8285422
Data columns (total 6 columns):
Keyword_ID       int64
Ad_group_ID      int64
Campaign_ID      int64
Account_ID       int64
Device_ID        int64
Match_type_ID    int64
dtypes: int64(6)
memory usage: 379.3 MB


In [8]:
train_set["RPC"] = train_set["Revenue"].values/train_set["Clicks"].values

In [9]:
train_set["RPC"].describe()

count    8.285423e+06
mean     5.113287e+01
std      7.113045e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.148236e+05
Name: RPC, dtype: float64

In [10]:
df_feature=train_set[feature_list]
train_X=df_feature.apply(lambda x : pd.factorize(x)[0])
train_X.astype(str).describe()

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID
count,8285423,8285423,8285423,8285423,8285423,8285423
unique,487981,269480,2927,16,3,3
top,28,240,3,7,0,0
freq,5188,8111,325658,2300896,3980401,3754784


In [11]:
new_train = train_set
new_train[feature_list] = train_X

# for i in feature_list:
#     new_train[i] = train_X[i].astype('category')
# new_train = new_train.drop(['Date','Revenue','Clicks','Conversions'],axis=1)
# X = new_train[feature_list].drop(['Account_ID','Device_ID','Match_type_ID'],axis=1)

# list(new_train)

# new_train.to_csv("train.csv",sep=',',index=False)
# predict和 train 需要用同一套label 不然肯定出错 
# TODO merge label 拆分

In [12]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8285423 entries, 0 to 8285422
Data columns (total 11 columns):
Date             object
Keyword_ID       int64
Ad_group_ID      int64
Campaign_ID      int64
Account_ID       int64
Device_ID        int64
Match_type_ID    int64
Revenue          float64
Clicks           int64
Conversions      int64
RPC              float64
dtypes: float64(2), int64(8), object(1)
memory usage: 695.3+ MB


In [72]:
X = new_train[['Keyword_ID','Ad_group_ID','Campaign_ID','Account_ID']]
y = new_train[['RPC']]

In [64]:
rs = ShuffleSplit(n_splits=5, test_size=.4)
i = 1
clf = LinearRegression()
for train_index, validate_index in rs.split(X,y):
    x_train = X.iloc[train_index]
    y_train = np.ravel(y.loc[train_index])
    x_validate = X.iloc[validate_index]
    y_validate = np.ravel(y.loc[validate_index])
    
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_validate)
    print("Iter",i)
    print("Error",np.sqrt(metrics.mean_squared_error(y_validate, y_pred)))
    print("Score",clf.score(x_validate,y_validate))
    i +=1

Iter 1
Error 733.220488254
Score 0.000277917548473
Iter 2
Error 762.928518717
Score 0.000279769267941
Iter 3
Error 748.68749086
Score 0.000274842967668
Iter 4
Error 717.704806746
Score 0.000314139451022
Iter 5
Error 691.25895103
Score 0.000348050328515


In [73]:
rs = ShuffleSplit(n_splits=2, test_size=.4)
i = 1
clf = SGDRegressor()
for train_index, validate_index in rs.split(X,y):
    x_train = X.iloc[train_index]
    x_validate = X.iloc[validate_index]
    y_train = np.ravel(y.loc[train_index])
    y_validate = np.ravel(y.loc[validate_index])

    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_validate)
    print("Iter",i)
    print("Error",np.sqrt(metrics.mean_squared_error(y_validate, y_pred)))
    print("Score",linreg.score(x_validate,y_validate))
    i +=1

(3314170, 4) (4971253, 4) 3314170
Iter 1
Error 5.38604019055e+17
Score 0.00035875881615
(3314170, 4) (4971253, 4) 3314170
Iter 2
Error 9.30102978366e+17
Score 0.00035732815366
